In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

# Fonction pour charger les données
def load_data():
    transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    train_dataset = ImageFolder('../data/Skin_cancer_ISIC/Train', transform=transform)
    test_dataset = ImageFolder('../data/Skin_cancer_ISIC/Test', transform=transform)
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)
    
    return train_loader, test_loader, len(train_dataset.classes)

# Initialiser le modèle (ResNet50 pré-entraîné)
def initialize_model(num_classes):
    model = models.resnet50(pretrained=True)
    
    # Geler tous les paramètres
    for param in model.parameters():
        param.requires_grad = False
    
    # Remplacer la dernière couche fully connected
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)
    
    # Optionnel : dégeler la dernière couche convolutionnelle
    for param in model.layer4.parameters():
        param.requires_grad = True
    
    return model

# Fonction d'évaluation du modèle
def evaluate_model(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    return accuracy

# Fonction d'entraînement avec l'approche progressive (ACET)
def train_model(model, train_loader, test_loader, num_epochs=50):
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    model = model.to(device)
    
    criterion = nn.MSELoss()  # Utilisation de MSE pour les cibles progressives
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
    
    epsilon = 1e-4  # Seuil d'équilibre
    target_multipliers = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]  # Cibles progressives
    epochs_per_multiplier = num_epochs // len(target_multipliers)  # Nombre d'époques pour chaque multiplicateur

    for t in target_multipliers:
        print(f"\nTraining with target multiplier: {t}")
        for epoch in range(epochs_per_multiplier):
            model.train()
            running_loss = 0.0
            correct = 0
            total = 0
            
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                
                optimizer.zero_grad()
                outputs = model(inputs)
                
                # Cibles progressives
                progressive_target = torch.zeros(labels.size(0), model.fc.out_features, device=device)
                progressive_target.scatter_(1, labels.unsqueeze(1), t)  # Application de t comme multiplicateur progressif
                
                loss = criterion(outputs, progressive_target)
                
                if loss.item() > epsilon:  # Mise à jour des poids si en dehors de l'équilibre
                    loss.backward()
                    optimizer.step()

                running_loss += loss.item()

                # Calcul de la précision pour cette époque
                pred = outputs.argmax(dim=1, keepdim=True)
                correct += pred.eq(labels.view_as(pred)).sum().item()
                total += labels.size(0)
            
            train_loss = running_loss / len(train_loader)
            train_accuracy = 100. * correct / total

            # Évaluation sur le test set après chaque époque
            test_accuracy = evaluate_model(model, test_loader, device)
            
            print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
                  f'Test Accuracy: {test_accuracy:.2f}%')

def main():
    train_loader, test_loader, num_classes = load_data()
    model = initialize_model(num_classes)
    train_model(model, train_loader, test_loader)

if __name__ == '__main__':
    main()


/Users/vpro/opt/anaconda3/envs/Torch_Info_theory/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/vpro/opt/anaconda3/envs/Torch_Info_theory/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



Training with target multiplier: 0.1
Epoch 1, Train Loss: 0.0227, Train Accuracy: 30.73%, Test Accuracy: 31.36%
Epoch 2, Train Loss: 0.0008, Train Accuracy: 54.94%, Test Accuracy: 33.05%
Epoch 3, Train Loss: 0.0008, Train Accuracy: 58.64%, Test Accuracy: 36.44%
Epoch 4, Train Loss: 0.0008, Train Accuracy: 60.21%, Test Accuracy: 43.22%
Epoch 5, Train Loss: 0.0007, Train Accuracy: 62.44%, Test Accuracy: 40.68%

Training with target multiplier: 0.2
Epoch 1, Train Loss: 0.0030, Train Accuracy: 60.83%, Test Accuracy: 48.31%
Epoch 2, Train Loss: 0.0027, Train Accuracy: 64.58%, Test Accuracy: 39.83%
Epoch 3, Train Loss: 0.0026, Train Accuracy: 65.83%, Test Accuracy: 42.37%
Epoch 4, Train Loss: 0.0025, Train Accuracy: 66.90%, Test Accuracy: 42.37%
Epoch 5, Train Loss: 0.0025, Train Accuracy: 68.51%, Test Accuracy: 45.76%

Training with target multiplier: 0.3
Epoch 1, Train Loss: 0.0057, Train Accuracy: 68.87%, Test Accuracy: 40.68%
Epoch 2, Train Loss: 0.0055, Train Accuracy: 69.54%, Test Acc